In [1]:
# --- 0. 安装必要的库 ---
# LDA需要 'gensim'
# 交互式可视化需要 'pyLDAvis'
!pip install -q gensim pyLDAvis

# --- 1. 导入库 ---
import pandas as pd
import numpy as np
import matplotlib.font_manager as fm # (pyLDAvis不一定使用,但保留)
import os
import sys
import re

# 导入 gensim
try:
    from gensim.corpora import Dictionary
    from gensim.models import LdaModel
except ImportError:
    print("错误：无法导入 gensim。")
    print("请确保Kaggle Notebook已开启网络，并已成功运行 !pip install gensim。")
    sys.exit()

# 导入 pyLDAvis
try:
    import pyLDAvis
    import pyLDAvis.gensim_models as gensim_vis
    # 在Kaggle/Jupyter中启用可视化
    pyLDAvis.enable_notebook()
except ImportError:
    print("错误：无法导入 pyLDAvis。")
    print("请确保Kaggle Notebook已开启网络，并已成功运行 !pip install pyLDAvis。")
    sys.exit()

# --- 2. 设置字体和文件路径 (Kaggle路径) ---
# 尽管pyLDAvis是网页组件,我们仍然设置字体以防万一
input_dir = '/kaggle/input/fundings/'

font_path_chinese = os.path.join(input_dir, 'simkai.ttf')
font_path_english = os.path.join(input_dir, 'times.ttf')
data_file = os.path.join(input_dir, '2025-jiaoyubu-fenci.csv')

# 检查字体文件
if os.path.exists(font_path_chinese):
    fm.fontManager.addfont(font_path_chinese)
if os.path.exists(font_path_english):
    fm.fontManager.addfont(font_path_english)

# --- 3. 加载和准备数据 ---
print(f"正在从 '{data_file}' 加载数据...")
try:
    df = pd.read_csv(data_file)
except FileNotFoundError:
    print(f"错误: 数据文件 '{data_file}' 未找到。")
    print("请确保您的数据文件和字体文件已上传到Kaggle的 '/kaggle/input/fundings/' 数据集中。")
    sys.exit()

print("数据加载成功。")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 1.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 34.3 MB/s eta 0:00:00:00:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
tsfresh 0.21.0 requires scipy>=1.14.0; python_version >= "3.10", but you have scipy 1.13.1 which is incompatible.
dopamine-rl 4.1.2 requires gymnasium>=1.0.0, but you have gymnasium 0.29.0 which is incompatible.
imbalanced-learn 0.13.0 requires scikit-learn<2,>=1.3.2, but you have scikit-learn 1.2.2 which is incompatible.
plotnine 0.14.5 requires matplotlib>=3.8.0, but you have matplotlib 3.7.2 which is incompatible.
umap-learn 0.5.9.post2 requires scikit-learn>=1.6, but you have scikit-learn 1.2.2 which is incompatible.
mlxtend 0.23.4 requires scikit

In [2]:
target_discipline = '管理学'
management_df = df[df['学科门类'] == target_discipline].copy()

if management_df.empty:
    print(f"错误：未能在数据中找到学科门类 '{target_discipline}'。")
    sys.exit()

# --- 4. 文本预处理 (移除停用词) ---
print("正在准备 '管理学' 文本数据...")
texts_raw = management_df['项目名称分词'].dropna().apply(lambda x: str(x).split()).tolist()

# 定义常见的项目名称停用词
my_stop_words = set([
    '研究', '基于', '影响', '机制', '路径', '策略', '模式', 
    '对策', '探析', '视域', '赋能', '建构', '实证', '问题',
    '——'
])
print(f"将从文本中移除 {len(my_stop_words)} 个自定义停用词...")

texts_processed = [
    [word for word in doc if word not in my_stop_words]
    for doc in texts_raw
]

if not texts_processed:
    print("错误：移除停用词后所有文档均为空。")
    sys.exit()

print("停用词移除完成。")

# --- 5. 创建 Gensim 词典和语料库 ---
print("正在创建 Gensim 词典和语料库...")
id2word = Dictionary(texts_processed)
# 过滤词典：移除低频（<2）和高频（>0.5）词
id2word.filter_extremes(no_below=2, no_above=0.5)
corpus = [id2word.doc2bow(text) for text in texts_processed]

if not any(corpus):
    print("错误：语料库为空，可能是过滤条件过于严格。")
    sys.exit()

print(f"词典和语料库创建完成。词典大小 (过滤后): {len(id2word)}")

# --- 6. 训练最终的 LDA 模型 (K=10) ---
# --- 微调：主题数设置为 10 ---
num_topics = 10
print(f"正在训练最终的 LDA 模型, 主题数 K={num_topics}...")

lda_model = LdaModel(
    corpus=corpus,
    id2word=id2word,
    num_topics=num_topics,
    random_state=2025,
    passes=15, # 使用更多遍数进行最终训练
    chunksize=100
)
print("LDA 模型训练完成。")

# --- 7. 准备并保存 pyLDAvis 交互式可视化 ---
print("正在准备 pyLDAvis 交互式可视化 (这可能需要几秒钟)...")

# 这是将 gensim 模型转换为 pyLDAvis 所需数据的核心步骤
vis = gensim_vis.prepare(lda_model, corpus, id2word)

# 保存为 HTML 文件
output_filename = 'management_lda_k10_interactive.html'
try:
    pyLDAvis.save_html(vis, output_filename)
    print(f"交互式图表已成功保存为: {output_filename} (位于 /kaggle/working/ 目录)")
except Exception as e:
    print(f"保存交互式HTML时出错: {e}")

# --- 8. 在Kaggle Notebook中显示交互式图表 ---
print("\n" + "="*50)
print("正在Kaggle Notebook中显示交互式图表...")
print("="*50)

# 在Kaggle单元格的最后一行返回 'vis' 对象，它会自动渲染
vis

# --- 9. 打印每个主题下概率最高的10个主题词 ---
print("\n" + "="*50)
print(f"主题词分布 (Top 10 per topic) for '{target_discipline}'")
print("="*50)

for topic_id in range(num_topics):
    print(f"\n【主题 {topic_id}】")
    # 获取该主题下权重最高的10个词及其概率
    top_terms = lda_model.show_topic(topic_id, topn=10)
    for rank, (term, prob) in enumerate(top_terms, start=1):
        print(f"{rank:>2}. {term:<15s} 概率: {prob:.4f}")


正在准备 '管理学' 文本数据...
将从文本中移除 15 个自定义停用词...
停用词移除完成。
正在创建 Gensim 词典和语料库...
词典和语料库创建完成。词典大小 (过滤后): 435
正在训练最终的 LDA 模型, 主题数 K=10...
LDA 模型训练完成。
正在准备 pyLDAvis 交互式可视化 (这可能需要几秒钟)...
交互式图表已成功保存为: management_lda_k10_interactive.html (位于 /kaggle/working/ 目录)

正在Kaggle Notebook中显示交互式图表...

主题词分布 (Top 10 per topic) for '管理学'

【主题 0】
 1. 协同              概率: 0.0605
 2. 效应              概率: 0.0528
 3. 干预              概率: 0.0375
 4. 方法              概率: 0.0303
 5. 信息              概率: 0.0296
 6. 双刃剑             概率: 0.0278
 7. 技术              概率: 0.0271
 8. 目标              概率: 0.0267
 9. 机理              概率: 0.0234
10. 驱动              概率: 0.0230

【主题 1】
 1. 提升              概率: 0.0879
 2. 韧性              概率: 0.0817
 3. 评估              概率: 0.0512
 4. 城市              概率: 0.0507
 5. 价值              概率: 0.0410
 6. 政策              概率: 0.0316
 7. 优化              概率: 0.0311
 8. 数字              概率: 0.0271
 9. 服务              概率: 0.0250
10. 低空              概率: 0.0234

【主题 2】
 1. 治理              概率: 0.0532
 2. 企业     

In [3]:
target_discipline = '交叉学科/综合研究'
management_df = df[df['学科门类'] == target_discipline].copy()

if management_df.empty:
    print(f"错误：未能在数据中找到学科门类 '{target_discipline}'。")
    sys.exit()

# --- 4. 文本预处理 (移除停用词) ---
print("正在准备 '交叉学科/综合研究' 文本数据...")
texts_raw = management_df['项目名称分词'].dropna().apply(lambda x: str(x).split()).tolist()

# 定义常见的项目名称停用词
my_stop_words = set([
    '研究', '基于', '影响', '机制', '路径', '策略', '模式', 
    '对策', '探析', '视域', '赋能', '建构', '实证', '问题',
    '——'
])
print(f"将从文本中移除 {len(my_stop_words)} 个自定义停用词...")

texts_processed = [
    [word for word in doc if word not in my_stop_words]
    for doc in texts_raw
]

if not texts_processed:
    print("错误：移除停用词后所有文档均为空。")
    sys.exit()

print("停用词移除完成。")

# --- 5. 创建 Gensim 词典和语料库 ---
print("正在创建 Gensim 词典和语料库...")
id2word = Dictionary(texts_processed)
# 过滤词典：移除低频（<2）和高频（>0.5）词
id2word.filter_extremes(no_below=2, no_above=0.5)
corpus = [id2word.doc2bow(text) for text in texts_processed]

if not any(corpus):
    print("错误：语料库为空，可能是过滤条件过于严格。")
    sys.exit()

print(f"词典和语料库创建完成。词典大小 (过滤后): {len(id2word)}")

# --- 6. 训练最终的 LDA 模型 (K=10) ---
# --- 微调：主题数设置为 10 ---
num_topics = 10
print(f"正在训练最终的 LDA 模型, 主题数 K={num_topics}...")

lda_model = LdaModel(
    corpus=corpus,
    id2word=id2word,
    num_topics=num_topics,
    random_state=2025,
    passes=15, # 使用更多遍数进行最终训练
    chunksize=100
)
print("LDA 模型训练完成。")

# --- 7. 准备并保存 pyLDAvis 交互式可视化 ---
print("正在准备 pyLDAvis 交互式可视化 (这可能需要几秒钟)...")

# 这是将 gensim 模型转换为 pyLDAvis 所需数据的核心步骤
vis = gensim_vis.prepare(lda_model, corpus, id2word)

# 保存为 HTML 文件
output_filename = 'cross_lda_k10_interactive.html'
try:
    pyLDAvis.save_html(vis, output_filename)
    print(f"交互式图表已成功保存为: {output_filename} (位于 /kaggle/working/ 目录)")
except Exception as e:
    print(f"保存交互式HTML时出错: {e}")

# --- 8. 在Kaggle Notebook中显示交互式图表 ---
print("\n" + "="*50)
print("正在Kaggle Notebook中显示交互式图表...")
print("="*50)

# 在Kaggle单元格的最后一行返回 'vis' 对象，它会自动渲染
vis

# --- 9. 打印每个主题下概率最高的10个主题词 ---
print("\n" + "="*50)
print(f"主题词分布 (Top 10 per topic) for '{target_discipline}'")
print("="*50)

for topic_id in range(num_topics):
    print(f"\n【主题 {topic_id}】")
    # 获取该主题下权重最高的10个词及其概率
    top_terms = lda_model.show_topic(topic_id, topn=10)
    for rank, (term, prob) in enumerate(top_terms, start=1):
        print(f"{rank:>2}. {term:<15s} 概率: {prob:.4f}")


正在准备 '交叉学科/综合研究' 文本数据...
将从文本中移除 15 个自定义停用词...
停用词移除完成。
正在创建 Gensim 词典和语料库...
词典和语料库创建完成。词典大小 (过滤后): 764
正在训练最终的 LDA 模型, 主题数 K=10...
LDA 模型训练完成。
正在准备 pyLDAvis 交互式可视化 (这可能需要几秒钟)...
交互式图表已成功保存为: cross_lda_k10_interactive.html (位于 /kaggle/working/ 目录)

正在Kaggle Notebook中显示交互式图表...

主题词分布 (Top 10 per topic) for '交叉学科/综合研究'

【主题 0】
 1. 旅游              概率: 0.0499
 2. 系统              概率: 0.0369
 3. 转型              概率: 0.0265
 4. 农业              概率: 0.0237
 5. 活化              概率: 0.0231
 6. 利用              概率: 0.0228
 7. 媒体              概率: 0.0212
 8. 调控              概率: 0.0211
 9. 整理              概率: 0.0207
10. 逻辑              概率: 0.0194

【主题 1】
 1. 驱动              概率: 0.0639
 2. 乡村              概率: 0.0580
 3. 传统              概率: 0.0549
 4. 重构              概率: 0.0322
 5. 空间              概率: 0.0315
 6. 儿童              概率: 0.0273
 7. 经济              概率: 0.0273
 8. 演变              概率: 0.0260
 9. 国家              概率: 0.0238
10. 公共              概率: 0.0210

【主题 2】
 1. 健康              概率: 0.0456
 2. 